# Analysis of IndieDAO

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statistics
import ast
import requests

### DAO queries and data retrieval 

In [ ]:
# Production hub
URL = "https://hub.snapshot.org/graphql"
# "id" of the Snapshot space, i.e. IndieDAO space
DAO = "indiedao.eth"

Save proposals information

In [ ]:
def get_all_proposals(dao, date):
  QUERY_VOTES = f"""query Proposals {{
  proposals(
    where: {{space_in: "{dao}", created_gt: {date}}},
    first: 1000,
    orderBy: "created",
    orderDirection: asc
  ) {{
    id
    created
    title
    votes
    type
    state
    author
    scores
    scores_total
    choices
    end
  }}
}}
"""
  return QUERY_VOTES

def set_all_proposals_info(dao):
  date = 0
  list_dic_proposals = []
  while True:
    props = requests.post(URL, json={'query': get_all_proposals(dao, date)}).json()["data"]["proposals"]

    list_dic_proposals += props
    date = max(list(map(lambda x: x["created"], props)))

    if len(props) < 1000:
      break

  df_proposals = pd.DataFrame.from_dict(list_dic_proposals, orient='columns')
  df_proposals['created'] = pd.to_datetime(df_proposals['created'], unit='s')
  df_proposals['end'] = pd.to_datetime(df_proposals['end'], unit='s')
  df_proposals['author'] = df_proposals['author'].str.lower()

  return df_proposals


df_proposals = set_all_proposals_info(DAO)
df_proposals.to_csv("proposals.csv")
print(len(df_proposals["id"]), "proposals read")

Save the voting information

In [ ]:
def get_all_votes(dao, date):
  QUERY_VOTES = f"""query Votes {{
    votes(
      where: {{space_in: "{dao}", created_gt: {date} }},
      first: 1000,
      orderBy: "created",
      orderDirection: asc
    ) {{
      id
      created
      voter
      vp
      choice
      proposal {{
          id
          title
      }}
    }}
  }}
  """
  return QUERY_VOTES

def set_votes_info(dao):
  date = 0
  list_dict_results = []
  while True:

    votes = requests.post(URL, json={'query': get_all_votes(dao, date)}).json()["data"]["votes"]
    list_dict_results += votes
    date = max(list(map(lambda x: x["created"], votes)))

    if len(votes) < 1000:
      break

  df_votes = pd.DataFrame.from_dict(list_dict_results, orient='columns')
  df_votes['created'] = pd.to_datetime(df_votes['created'], unit='s')
  df_votes['voter'] = df_votes['voter'].str.lower()
  df_votes["choice"] = df_votes["choice"].apply(lambda val: val[0] if (type(val) == list) else val)
  return df_votes


df_votes = set_votes_info(DAO)
df_votes.to_csv("votes.csv")
print(len(df_votes["id"]), "votes read")

Load the files with the DAO proposal and voting information.

In [ ]:
proposals = pd.read_csv('proposals.csv')
votes = pd.read_csv('votes.csv')

Data cleaning.

In [ ]:
# Assuming votes['proposal'] is a list of strings that look like dictionaries
proposal_list = votes['proposal']

# Convert each string in the list to a dictionary
proposal_dicts = [ast.literal_eval(proposal_str) for proposal_str in proposal_list]

# 'id' of each proposal
proposal_ids = [proposal_dict['id'] for proposal_dict in proposal_dicts]

# 'title' of each proposal
proposal_titles = [proposal_dict['title'] for proposal_dict in proposal_dicts]
votes['prop_id'] = proposal_ids
votes['prop_title'] = proposal_titles
#votes.drop('proposal', axis=1, inplace=True)

# Lowercase all data
votes['id'] = votes['id'].str.lower()
votes['voter'] = votes['voter'].str.lower()
votes['prop_id'] = votes['prop_id'].str.lower()

proposals['id'] = proposals['id'].str.lower()
proposals['author'] = proposals['author'].str.lower()

# Time period to be studied
ini = min(votes["created"])
fin = max(votes["created"])
allMonths = pd.date_range(start=ini, end=fin, freq=pd.DateOffset(months=1)).strftime("%Y-%m")
allMonthsNew = pd.date_range(start=ini, end=fin, freq=pd.DateOffset(months=1)).strftime("%Y-%b")

# Auxiliary structure used to display graphics in a more aesthetic way
DateAxis = []
for k in allMonthsNew:
  sp = k.split("-")
  if sp[1] != "Jan":
    DateAxis.append(sp[1])
  else:
    DateAxis.append(sp[1]+"\n"+sp[0])


Sort proposals by creation date to review them in order

In [ ]:
auxTuple = zip(proposals['id'], proposals['created'])
SortedPropsIDs = sorted(auxTuple, key=lambda date : date[1])

Some voters had to transition from one account to another (see the corresponding publication)
Thus, the addresses are unified:

- 0x7D4F0B2d660aa625B7aA7C6B72a7c0C884cab883 Old ID
- 0x62F77E12B8835f13ec37Fc7a9763A7efB87B6615 New ID

In [ ]:
votes['voter'] = votes['voter'].replace({"0x7d4f0b2d660aa625b7aa7c6b72a7c0c884cab883": "0x62f77e12b8835f13ec37fc7a9763a7efb87b6615"})
proposals['author'] = proposals['author'].replace({"0x7d4f0b2d660aa625b7aa7c6b72a7c0c884cab883": "0x62f77e12b8835f13ec37fc7a9763a7efb87b6615"})

- 0x6107e341e1f93af3e32fde1a104bd39fbad1e30e Old ID
- 0x10854be0c17346a1197160afd06d79df6006ecb9 New ID

In [ ]:
votes['voter'] = votes['voter'].replace({"0x6107e341e1f93af3e32fde1a104bd39fbad1e30e": "0x10854be0c17346a1197160afd06d79df6006ecb9"})
proposals['author'] = proposals['author'].replace({"0x6107e341e1f93af3e32fde1a104bd39fbad1e30e": "0x10854be0c17346a1197160afd06d79df6006ecb9"})

## Analysis of votes in the DAO

Votes per proposal

In [ ]:
num_voters = votes['voter'].nunique()
votes_series = proposals['votes']

print(f"Max number of votes in a proposal: {votes_series.max()}, {votes_series.max()/num_voters*100:.2f}%")
print(f"Min number of votes in a proposal: {votes_series.min()}, {votes_series.min()/num_voters*100:.2f}%")
print(f"Median number of votes: {votes_series.median()}, {votes_series.median()/num_voters*100:.2f}%")
print(f"Mean number of votes: {votes_series.mean():.2f}, {votes_series.mean()/num_voters*100:.2f}%")


Votes per voter

In [ ]:
num_props = len(proposals['id'])

votes_gr = votes.groupby('voter')['id'].count()

max_v = votes_gr.max()
min_v = votes_gr.min()
med_v = statistics.median(votes_gr)
mean_v = float(statistics.mean(votes_gr))

print(f"Max number of votes for a voter: {max_v}, {max_v/num_props*100:.2f}%")
print(f"Min number of votes for a voter: {min_v}, {min_v/num_props*100:.2f}%")
print(f"Median number of votes: {med_v}, {med_v/num_props*100:.2f}%")
print(f"Mean number of votes: {mean_v:.2f}, {mean_v/num_props*100:.2f}%")

## Analysis of Voting Power in the DAO

In [ ]:
# sum of the median vp values of the voters
vp = [statistics.median(votes[votes['voter'] == voter]['vp']) for voter in set(votes['voter'])]
total_median_vp = sum(vp)

print(f"Sum of the median vp values of the voters: {total_median_vp:.2f}")

# Proposals' VP metrics (display with two decimal places)
scores = proposals['scores_total']

print(f"Max VP in a proposal: {scores.max():.2f}, {scores.max()/total_median_vp*100:.2f}%")
print(f"Min VP in a proposal: {scores.min():.2f}, {scores.min()/total_median_vp*100:.2f}%")
print(f"Median VP in a proposal: {statistics.median(scores):.2f}, {statistics.median(scores)/total_median_vp*100:.2f}%")
print(f"Mean VP in a proposal: {statistics.mean(scores):.2f}, {statistics.mean(scores)/total_median_vp*100:.2f}%")

# Voters' VP (median)
vp_gr = votes.groupby('voter')['vp'].median()

max_vp = vp_gr.max()
min_vp = vp_gr.min()
med_vp = statistics.median(vp_gr)
mean_vp = statistics.mean(vp_gr)

print(f"Max vp for a voter (median): {max_vp:.2f}, {max_vp/total_median_vp*100:.2f}%")
print(f"Min vp for a voter (median): {min_vp:.2f}, {min_vp/total_median_vp*100:.2f}%")
print(f"Median vp for a voter (median): {med_vp:.2f}, {med_vp/total_median_vp*100:.2f}%")
print(f"Mean vp for a voter (median): {mean_vp:.2f}, {mean_vp/total_median_vp*100:.2f}%")

## Analysis of the results of the proposals

In [ ]:
secundada = 0
nosecundada = 0
novotos = 0
unanimidadConAutor = 0 
unanimidadSinAutor = 0 
unanimidad = 0

for prop_id in proposals['id']:
    autor = list(proposals[proposals['id'] == prop_id]['author'])[0]
    
    autor_aux = votes['voter'] == autor
    prop_aux = np.array(votes['prop_id']) == prop_id 
    filter = votes[np.logical_and(prop_aux , autor_aux)]['created']
    
    num_votos = list(proposals[proposals['id'] == prop_id]['votes'])[0]
    if num_votos == 0:
        novotos +=1
    
    elif len(filter) > 0: 
        opt_autor = votes[np.logical_and(prop_aux , autor_aux)]['choice'].to_list()[0]        
        # winner option
        opt_win = votes[votes['prop_id'] == prop_id].groupby('choice')['vp'].sum().idxmax()        
        
        if opt_win == opt_autor:
            secundada += 1 
            
            aux = votes[votes['prop_id'] == prop_id]
            aux2 = aux[aux['choice']==opt_win]['id'].count()
            if num_votos == aux2:
                unanimidadConAutor +=1                
                
        else:       
            nosecundada +=1

    else:
        aux = votes[votes['prop_id'] == prop_id]
        aux2 = aux[aux['choice']==1]['id'].count()
        
        if num_votos == aux2:
                unanimidadSinAutor +=1        

        
print(f"The option voted for by the author wins: {secundada}")
print(f"The option voted for by the author does not win: {nosecundada}")
print(f"Proposals without votes: {novotos}")
print(f"Unanimous proposals (the author does not vote): {unanimidadSinAutor}")
print(f"Unanimous proposals (same opinion as the author): {unanimidadConAutor}")

In [ ]:
vp = []

for prop in proposals['id']:
    opt_win = votes[votes['prop_id'] == prop].groupby('choice')['vp'].sum().idxmax()
    vp_aux = votes[votes['prop_id'] == prop]
    vp_aux2 = vp_aux[vp_aux['choice']==opt_win]['vp'].sum()

    a = list(proposals[proposals['id']==prop]['scores_total'])[0]
    vp.append(vp_aux2 / a * 100)
    

print(f"Average VP of the winning option: {statistics.mean(vp):.2f}")

In [ ]:
votos = []

for prop in proposals['id']:
    opt_win = votes[votes['prop_id'] == prop].groupby('choice')['vp'].sum().idxmax()
    vp_aux = votes[votes['prop_id'] == prop]
    vp_aux2 = vp_aux[vp_aux['choice']==opt_win]['id'].count()

    a = list(proposals[proposals['id']==prop]['votes'])[0]
    votos.append(vp_aux2 / a * 100)
    
        
print(f"Average votes for winning option: {statistics.mean(votos):.2f}")

# Analysis of inequality in the DAO

In [ ]:
# voters who together account for > 50% of the VP 

totalVP = 0
vps = []
for voter in set(votes['voter']):
  sum_vp = votes[votes['voter']==voter]['vp'].median()
  totalVP += sum_vp
  vps.append(sum_vp)

ordenado = sorted(vps, reverse = True)

sumi = 0
for i in range(len(ordenado)):
  sumi += ordenado[i]
  if sumi > totalVP/2:
    res = i+1
    print(f"Number of voters that together hold > 50% of the DAO VP: {res}")
    break

print(f"Percentage: {res/len(ordenado)*100:.2f}%")

Calculation of the Gini coefficient (per voter) to measure inequality among VPs

In [ ]:
#GINI 
def gini(x):
    total = 0
    for i, xi in enumerate(x[:-1], 1):
        total += np.sum(np.abs(xi - x[i:]))
    return total / (len(x)**2 * np.mean(x))

VP_gruped_per_voter = votes.groupby(["voter"])["vp"]
data = VP_gruped_per_voter.median()
print(f"Gini coefficient: {gini(data):.2f}")

Plot of the Gini coefficient for the different proposals to observe inequality in the distribution of VPs

In [ ]:
#gini per proposal
plt.rcParams["figure.figsize"]=(16,10)
data = {}
cadena = "P"
i = 1

for prop_id, date in SortedPropsIDs:
  if list(proposals[proposals['id'] == prop_id]['scores_total'])[0] != 0:
    prop_aux = np.array(votes['prop_id']) == prop_id
    total_vp = votes[ prop_aux ]['vp']
    data["P"+str(i)] = gini(np.array(total_vp))
  i+=1

plt.bar(list(data.keys()), list(data.values()))
#plt.axhline(0.9)
plt.ylim(0,1)
plt.xticks(rotation=45)
plt.xlabel('Proposal', fontsize=25)
plt.ylabel('Gini coefficient', fontsize=25)

plt.xticks(fontsize=12)
plt.yticks(fontsize=20)
plt.yticks(np.arange(0, 1.1, 0.1))
plt.savefig("gini_prop.pdf", format="pdf", bbox_inches="tight")

plt.show()

In [ ]:
print(f"Average Gini values for the different proposals: {statistics.mean(data.values()):.2f}")
print()
print(f"Median of Gini values for the different proposals: {statistics.median(data.values()):.2f}")

Representation of the Lorez curve to observe VP inequality in the DAO

In [ ]:
def lorenz_curve(X):
  X_lorenz = X.cumsum() / X.sum()
  X_lorenz = np.insert(X_lorenz, 0, 0)
  X_lorenz[0], X_lorenz[-1]

  fig, ax = plt.subplots(figsize=[6,6])
  ## scatter plot of Lorenz curve
  ax.scatter(np.arange(X_lorenz.size)/(X_lorenz.size-1), X_lorenz, 
           s=80, facecolors='none', edgecolors='darkorange')
  ## line plot of equality
  ax.plot([0,1], [0,1], color='k')
  plt.axhline(y=0.5, color='grey', lw=0.5)
  
  plt.ylabel("Cumulative voting power" , fontsize=20)
  plt.xlabel("Cumulative voters", fontsize=20)

  plt.xticks(fontsize=15)
  plt.yticks(fontsize=15)

  plt.savefig("lorenz.pdf", format="pdf", bbox_inches="tight")

  plt.show()



X = dict(votes.groupby(["voter"])["vp"].median())
X = np.array(sorted(list(X.values())))

print("Lorenz curve with VP median")
lorenz_curve(X)

# Analysis of voters with the highest VP (whales)

Observation of the 5 voters with the highest VP

In [ ]:
votes.loc[votes.groupby("voter")["vp"].idxmax()].nlargest(5, "vp")[["voter", "vp"]]

Number of votes cast by the 5 voters with the highest VP

In [ ]:
for id in votes.loc[votes.groupby("voter")["vp"].idxmax()].nlargest(5, "vp")["voter"]:
    print("Voter", id, "voted", len(votes[votes['voter']==id]), "times.")

Range diagram of each voter's VP, sorted from lowest to highest average VP

In [ ]:
test = []
myDict = {}

for voter in set(votes['voter']):
    mymax = max(votes[votes['voter'] == voter]['vp'])
    mymin = min(votes[votes['voter'] == voter]['vp'])
    mymedian = statistics.median(votes[votes['voter'] == voter]['vp'])
    myDict[voter] = (mymax + 1, mymin + 1, mymedian + 1)
    test.append(mymedian)

ordenado = sorted(myDict.items(), key=lambda x: x[1][2])

x_values = range(len(ordenado))  # Valores originales de 0 a N-1

for i in x_values:
    plt.vlines(i, ordenado[i][1][1], ordenado[i][1][0], color='black', zorder=0)

plt.scatter(x_values, list(map(lambda p: p[1][2], ordenado)), zorder=5)

plt.xticks(ticks=x_values, labels=range(1, len(ordenado) + 1), fontsize=15)  # Etiquetas de 1 a N
plt.yticks(fontsize=15)

plt.xlabel("Voters", fontsize=20)
plt.ylabel("Voting power", fontsize=20)

plt.ylim(-0.1, max(map(lambda p: p[1][0], ordenado)) + 10000)  # Eje Y empieza en 0 y se ajusta al máximo de vp

plt.grid()
plt.axvline(x=23.5, color='b', label='50% of estimated VP')

plt.show()


# Analysis of the authors of the proposals

We plot the distribution of proposals over time.

- Green: author votes for the winning option

- Gray: no one votes

- Blue: author does not vote

- Red: author votes for the losing option

In [ ]:
azul = {}
gris = {}
verde = {}
rojo = {}
for date in allMonthsNew:
  azul[date] = 0
  gris[date] = 0
  verde[date] = 0
  rojo[date] = 0


for prop_id in proposals['id']:
  autor = list(proposals[proposals['id'] == prop_id]['author'])[0]

  autor_aux = votes['voter'] == autor
  prop_aux = np.array(votes['prop_id']) == prop_id

  date = list(pd.to_datetime(proposals[proposals['id'] == prop_id]['created']))[0].strftime('%Y-%b')
    
  num_votos = list(proposals[proposals['id'] == prop_id]['votes'])[0]
  filter = votes[ np.logical_and(prop_aux , autor_aux)]['created']
  
  #grey: no one votes
  if num_votos == 0:
    gris[date] += 1  
  #blue: author does not vote
  elif len(filter) == 0:
    if date in azul:
      azul[date] += 1

  else:
    opt_autor = votes[ np.logical_and(prop_aux , autor_aux)]['choice'].to_list()[0]  
    opt_winner = votes[votes['prop_id'] == prop_id].groupby('choice')['vp'].sum().idxmax() 

    #green: the option the author voted wins
    if opt_winner == opt_autor :
      verde[date] += 1
    #red
    else:
      rojo[date] +=1

y1 = np.array(list(gris.values()))
y2 = np.array(list(azul.values()))
y3 = np.array(list(rojo.values()))


plt.bar(gris.keys(), gris.values(), color='silver', label='No votes')
plt.bar(azul.keys(), azul.values(), bottom=y1, color='darkturquoise', label='Author doesnt vote')
plt.bar(rojo.keys(), rojo.values(), bottom=y1 + y2, color='red', label='Authors option doesnt win')
plt.bar(verde.keys(), verde.values(), bottom=y1+y2+y3, color='limegreen', label='Authors option does win')
plt.legend()
plt.title("Proposals over time")
plt.ylabel("Number of proposals")
plt.xticks(rotation=90)
plt.tight_layout()

We plot the number of votes for each proposal.

- Blue: the author has not voted

- Dark green: the author's vote

- Green: other members' votes for the same option as the author.

- Red: other members' votes for an option different from the author's.

In [ ]:
azul = {}
verde = {}
rojo = {}
rosa = {}
gris={}
i=1
for prop in proposals['id']:
  azul["P"+str(i)] = 0
  verde["P"+str(i)] = 0
  rojo["P"+str(i)] = 0
  rosa["P"+str(i)] = 0
  gris["P"+str(i)] = 0
  i+=1

i=1
for prop_id, nousar in SortedPropsIDs:
  # print("P"+str(i))
  # print(prop_id)
  autor = list(proposals[proposals['id'] == prop_id]['author'])[0]

  autor_aux = votes['voter'] == autor
  prop_aux = np.array(votes['prop_id']) == prop_id

    
  num_votos = list(proposals[proposals['id'] == prop_id]['votes'])[0]
  filter = votes[ np.logical_and(prop_aux , autor_aux)]['created']
  
  #grey: no one votes, not represented in the plot
  if num_votos == 0:
    gris["P"+str(i)] += 1
  #Author does not vote
  elif len(filter) == 0:
    azul["P"+str(i)] += num_votos

  else:
    #Author votes
    opt_autor = votes[ np.logical_and(prop_aux , autor_aux)]['choice'].to_list()[0]    
    
    #num votes - author option
    n = votes[votes['prop_id']==prop_id]
    n = n[n['choice']==opt_autor]['id'].count()
    #green -> votes count = votes recieved by the option the author chose
    verde["P"+str(i)] += n -1
    #dark green -> author's vote
    rosa["P"+str(i)] += 1
    #red
    rojo["P"+str(i)] += num_votos - n
  i+=1

y1 = np.array(list(azul.values()))
y2 = np.array(list(rojo.values()))
y3 = np.array(list(verde.values()))

plt.bar(azul.keys(), azul.values(), color='darkturquoise', label='Author doesnt vote')
plt.bar(rojo.keys(), rojo.values(), bottom=y1, color='red', label='Votes against the authors option')
plt.bar(verde.keys(), verde.values(), bottom=y1+y2, color='limegreen', label='Votes for the authors option')
plt.bar(rosa.keys(), rosa.values(), bottom=y1+y2+y3, color='darkgreen', label='Authors vote')

plt.xticks(rotation=90)
plt.title('Number of votes per proposal')


plt.show()

We plot the VP for each proposal.

- Blue: the author has not voted

- Dark green: the author's vote

- Green: other members' votes for the same option as the author.

- Red: other members' votes for an option different from the author's.

In [ ]:
azul = {}
verde = {}
rojo = {}
rosa = {}
gris={}
i=1
for prop in proposals['id']:
  azul["P"+str(i)] = 0
  verde["P"+str(i)] = 0
  rojo["P"+str(i)] = 0
  rosa["P"+str(i)] = 0
  gris["P"+str(i)] = 0
  i+=1

i=1
for prop_id, nousar in SortedPropsIDs:
  autor = list(proposals[proposals['id'] == prop_id]['author'])[0]

  autor_aux = votes['voter'] == autor
  prop_aux = np.array(votes['prop_id']) == prop_id
    
  num_votos = list(proposals[proposals['id'] == prop_id]['votes'])[0]
  filter = votes[ np.logical_and(prop_aux , autor_aux)]['created']

  total_vp = list(proposals[proposals['id'] == prop_id]['scores_total'])[0]
  
  # no votes
  if num_votos == 0:
    gris["P"+str(i)] += 1  
  # author does not vote
  elif len(filter) == 0:
    # all votes are represented by blue colour
    azul["P"+str(i)] += total_vp

  else:
    # The author votes
    opt_autor = votes[ np.logical_and(prop_aux , autor_aux)]['choice'].to_list()[0]  
    n = votes[votes['prop_id']==prop_id]
    n = n[n['choice']==opt_autor]['vp'].sum()
    
    vp_autor = votes[ np.logical_and(prop_aux , autor_aux)]['vp'].to_list()[0] 
    
    #green
    verde["P"+str(i)] += n - vp_autor
    rosa["P"+str(i)] += vp_autor
    
    rojo["P"+str(i)] += total_vp - n
  i+=1

y1 = np.array(list(azul.values()))
y2 = np.array(list(rojo.values()))
y3 = np.array(list(verde.values()))

plt.bar(azul.keys(), azul.values(), color='darkturquoise', label='Author doesnt vote')
plt.bar(rojo.keys(), rojo.values(), bottom=y1, color='red', label='VP against the authors option')
plt.bar(verde.keys(), verde.values(), bottom=y1+y2, color='limegreen', label='VP for the authors option')
plt.bar(rosa.keys(), rosa.values(), bottom=y1+y2+y3, color='darkgreen', label='Authors VP')

plt.xticks(rotation = 45)

plt.ylabel("Voting power")

ax = plt.gca()
ax2 = ax.twinx()
val = 180001/total_median_vp*100
ax2.set_ylabel("Percentage of total median voting power")



plt.title('Voting power per proposal')
plt.show()

# Other analyses

We represent the number of voters accumulated over time in the DAO.

In [ ]:
myDict = {}
aux=0
for date in allMonths:
  for voter in set(votes['voter']):
    date_aux = min(pd.to_datetime(votes[votes['voter'] == voter]['created']).dt.to_period('M').astype(str))
    if date == date_aux:
      aux += 1
  myDict[date] = aux

fig, ax = plt.subplots()
ax.bar(list(myDict.keys()), list(myDict.values()))
plt.xticks(range(len(DateAxis)), DateAxis)

plt.title('Cumulative new voters')
plt.show()


We represent the correlation between the median VP and the total number of votes cast by each voter using a scatter plot.

In [ ]:
VP_gruped_per_voter = votes.groupby(["voter"])["vp"]
VP_median_per_voter = VP_gruped_per_voter.median()

votes_per_voter = votes.groupby(["voter"])["id"].count()


plt.style.use('bmh')
plt.xlim(-4000, 260000)
plt.yticks(np.arange(0, 47, step=5))
plt.scatter(VP_median_per_voter, votes_per_voter, alpha=0.5, s=100)
plt.xlabel('Median voting power', fontsize=20)
plt.ylabel('Total number of votes', fontsize=20)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)

plt.savefig("votosVPx.pdf", format="pdf", bbox_inches="tight")

plt.show()

For all proposals, we represent the percentage of votes and VP of the winning option.

In [ ]:
vp = {}
nvotos = {}
x=1


for prop in proposals['id']:
    opt_winner = votes[votes['prop_id'] == prop].groupby('choice')['vp'].sum().idxmax()  

    a = votes[votes['prop_id'] == prop]
    vpWin = a[a['choice']==opt_winner]['vp'].sum()
    
    vpTotal = list(proposals[proposals['id']==prop]['scores_total'])[0]
    
    porcentaje = vpWin * 100 / vpTotal
    
    vp[ "P"+str(x)] = porcentaje
    x+=1
  
aux = {}
for prop in proposals['id']:  
    opt_winner = votes[votes['prop_id'] == prop].groupby('choice')['vp'].sum().idxmax()    
    a = votes[votes['prop_id'] == prop]
    num_votosWin = a[a['choice']==opt_winner]['id'].count()
    
    num_votos = list(proposals[proposals['id']==prop]['votes'])[0]
    
    perc = num_votosWin * 100 / num_votos
    aux[prop] = perc

y = 1
for prop, nousar in SortedPropsIDs:
  if prop in aux:
    nvotos["P"+str(y)] = aux[prop]
  y+=1
  

plt.style.use('bmh')


x_axis = np.arange(len(list(map(lambda p: p[0], nvotos))))
fig = plt.figure()
ax = plt.gca()


ax.bar( x_axis-0.2, list(nvotos.values()), width=0.4, align='center', color='tab:orange', label='Votes')
plt.xticks(x_axis, list(nvotos.keys()), rotation=45)

ax.tick_params(axis='y')

ax.bar(  x_axis+0.2, list(vp.values()), width=0.4, align='center', color='tab:blue', label='Voting power')
plt.legend(fontsize=20, loc=3)

plt.axhline(y=50, label="50%", color='b')

plt.savefig("agree_bar.pdf", format="pdf", bbox_inches="tight")
plt.show()

We represent the relationship between the percentage of votes and the percentage of VP of the winning option using a scatter plot.

In [ ]:
x = []
y = []

for i in vp.items():
  for j in nvotos.items():
    if i[0] == j[0]:
      y.append(i[1])
      x.append(j[1])

plt.style.use('bmh')
plt.axes().set_aspect('equal')
plt.scatter(x, y, alpha=0.3, s=150)
plt.xlim(0, 100)
plt.ylim(0, 100)


plt.ylabel('Percentage of voting power', fontsize=20)
plt.xlabel('Percentage of votes', fontsize=20)

plt.xticks(fontsize=15)
plt.yticks(fontsize=15)

plt.savefig("agree.pdf", format="pdf", bbox_inches="tight")
plt.show()

We represent the relationship between the average VP and the number of votes for each DAO proposal.

In [ ]:
x = []
y = []
ids = []
i=1
for prop_id in proposals['id']:

  num_votos = proposals[proposals['id']==prop_id]['votes']
  vp = proposals[proposals['id'] == prop_id]['scores_total']

  x.append(num_votos)
  y.append(vp/num_votos)
  ids.append("P"+str(i))
  i+=1

plt.style.use('bmh')
plt.scatter(x, y, alpha=0.5, s=150)
plt.ylabel('Average voting power', fontsize=20)
plt.xlabel('Votes amount', fontsize=20)


plt.savefig("prop_vvp.pdf", format="pdf", bbox_inches="tight")